In [ ]:
!pip install gymnasium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 5.5 MB/s eta 0:00:00


In [ ]:
import gymnasium as gym
import numpy as np
import warnings
from collections import deque
warnings.filterwarnings('ignore')
import matplotlib.pyplot as pyplot
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential,Model
import random


In [ ]:
env =gym.make('CartPole-v1',render_mode =None)
env.reset()
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
print(state_size,action_size)
for e in range(20):
    observation = env.reset()
    for t in range(50):
        env.render()
        ob, rev, done, oth_info,_ = env.step(env.action_space.sample())#taking a random action
        #print(ob)
        #print(rev)
        if (done):
            print("Game Episode {}/{} High Score = {}".format(e,20,t))
            break
env.close()
print("All episodes are over")
# Seeing how the Cartpole game works
# We have reward -> negative and positive
# Done -> game over
# Location observations (coordinated of cart and pole along with tilt angles )

4 2
Game Episode 0/20 High Score = 12
Game Episode 1/20 High Score = 21
Game Episode 2/20 High Score = 10
Game Episode 3/20 High Score = 18
Game Episode 4/20 High Score = 10
Game Episode 5/20 High Score = 14
Game Episode 6/20 High Score = 10
Game Episode 7/20 High Score = 39
Game Episode 8/20 High Score = 13
Game Episode 9/20 High Score = 11
Game Episode 10/20 High Score = 12
Game Episode 11/20 High Score = 10
Game Episode 12/20 High Score = 13
Game Episode 13/20 High Score = 18
Game Episode 14/20 High Score = 23
Game Episode 15/20 High Score = 30
Game Episode 16/20 High Score = 19
Game Episode 17/20 High Score = 25
Game Episode 18/20 High Score = 24
Game Episode 19/20 High Score = 13
All episodes are over


In [ ]:
class Agent:
        # Exploration vs Exploitation
        # Exploration :- Good at the beginning helps us explore the action space well
        # Exploitation :- Becomes Good at end using the positive feedback form memory
    def __init__(self,state_size,action_size):
        self.state_size =state_size
        self.action_size = action_size
        self.memory = deque(maxlen = 2000)
        self.gamma = 0.95 # discounting factor

        self.epsilon = 1 # trust of our knowledge
        self.epsilon_decay = 0.995# the rate at which our epsilon value decreases
        self.epsilon_min = 0.01# the maximum possible trust we can have on our data
        self.learning_rate = 0.001
        self.model = self.create_model()

    def create_model(self):
        model = Sequential()
        model.add(Dense(24,input_dim = state_size,activation = 'relu'))
        model.add(Dense(24,activation = 'relu'))
        model.add(Dense(action_size,activation = 'linear'))
        model.compile(loss = 'mse',optimizer = 'adam')
        model.summary()
        return model

    def remember(self,state,action,reward,next_state,done):
      #Remembering past experiences in memory -- deque
        self.memory.append((state,action,reward,next_state,done))

    def act(self,state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def train(self,batch_size):
        minibatch = random.sample(self.memory,batch_size)# loads a random sample of batch size from memory to train
        for state,action,reward,next_state,done in minibatch:
            target = reward# if done the target is reward
            if not done:
              # if not game over target is the current reward + the next target's reward * Gamma value
               # (next rewards are taken at a discounted value)
               # Q(S,A) = learning_rate * (reward + gamma * an estimate of future maximum value)
                target = (reward + self.gamma * np.amax(self.model.predict(next_state)[0]))
            else:
              target = reward
            target_f = self.model.predict(state)# our current states maximum predicted value
            target_f[0][action] = target # the new parameters for out action

            self.model.fit(state,target_f,epochs = 1,verbose = 0)# fit the state and final targetted values in out dataset
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay # reduce value of our epsilon == increase the value of our decay


In [ ]:
n_episodes = 100
out_dir = 'Content/Cartpole/'


In [ ]:
#creating our agent from our agent class
agent = Agent(state_size,action_size)
done = False
batch_size = 32

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 24)                120       
                                                                 
 dense_1 (Dense)             (None, 24)                600       
                                                                 
 dense_2 (Dense)             (None, 2)                 50        
                                                                 
Total params: 770 (3.01 KB)
Trainable params: 770 (3.01 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
#Training our current model
for i in range(n_episodes):
  state = env.reset()
  state = state[0]# get the initial state values
  state = np.reshape(state,(1,state_size)) # transpose
  for t in range(500):
    action = agent.act(state)
    # Takes an action in our envioenment and get the next values of states
    next_state,reward,done,_,_ = env.step(action)
    #Evaluate our reward if not done if game is over then penalize the model
    reward = reward if not done else -10

    next_state = np.reshape(next_state,[1,state_size])
    #adding state action reward next_state and game over in our memory
    agent.remember(state,action,reward,next_state,done)
    # Updating the current state to the next state
    state = next_state

    if done:
      # if game is over the print the current episode , totle episode , our score and value of our confidence
      print("Episode: {}/{}, Score: {}, e: {}".format(i,n_episodes,t,agent.epsilon))
      break
    if len(agent.memory) > batch_size:
      # if our memory has elements equal to our batch size we train out model on the memory
      agent.train(batch_size)
    if e%50 == 0:
      # saving our model every 50 episodes
      agent.model.save(out_dir+'weights_{}.h5'.format(e))
env.close()

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 29ms/step
Episode: 18/100, Score: 214, e: 0.011094979641301777
1/1 [==============================] - 0s 33ms/step


In [ ]:
# Final Output -----> Episode: 18/100, Score: 214, e: 0.011094979641301777